In [160]:
import numpy as np
import pandas as pd
import re
import datetime
import doctest
doctest.testmod(verbose=True)

3 items had no tests:
    __main__
    __main__.scoreSwp
    __main__.scoreWave
0 tests in 3 items.
0 passed and 0 failed.
Test passed.


TestResults(failed=0, attempted=0)

# Code

In [161]:
data = 'https://www.ndbc.noaa.gov/data/realtime2/46029.spec'
article_read = pd.read_csv(data, sep='\s+', delimiter=' ', encoding="utf-8", skipinitialspace=True)

In [162]:
article_read.head()

,#YY,MM,DD,hh,mm,WVHT,SwH,SwP,WWH,WWP,SwD,WWD,STEEPNESS,APD,MWD
0,#yr,mo,dy,hr,mn,m,m,sec,m,sec,-,degT,-,sec,degT
1,2019,03,21,21,00,1.7,1.6,12.1,0.6,4.8,W,SSW,SWELL,7.2,280
2,2019,03,21,20,00,1.9,1.8,12.1,0.6,3.8,WSW,S,SWELL,7.4,245
3,2019,03,21,19,00,1.9,1.8,12.9,0.7,5.3,W,SW,SWELL,7.3,270
4,2019,03,21,18,00,1.9,1.8,10.8,0.6,3.7,W,SSW,SWELL,7.1,261


In [163]:
short_table = article_read.drop(article_read.iloc[:,3:6,], axis=1)

In [164]:
short_table = short_table.drop(['STEEPNESS', 'APD', 'MWD'], axis=1)

In [165]:
d = datetime.datetime.today().strftime('%Y-%m-%d')
d

'2019-03-21'

In [166]:
today_date = datetime.date.today()
today_date

datetime.date(2019, 3, 21)

In [167]:
short_table = short_table.rename(columns={'#YY': 'YY'})

In [168]:
date = (short_table.YY + '-' + short_table.MM + '-' + short_table.DD)
short_table['Date'] = date

In [169]:
data_table = short_table.drop(short_table.columns[[0,1,2]], axis=1)
data_table = data_table[data_table['Date']==d]

In [170]:
data_table['SwH'] = data_table.SwH.astype(float)
data_table['SwP'] = data_table.SwP.astype(float)
data_table['WWH'] = data_table.WWH.astype(float)
data_table['WWP'] = data_table.WWP.astype(float)

In [171]:
data_table.dtypes

SwH     float64
SwP     float64
WWH     float64
WWP     float64
SwD      object
WWD      object
Date     object
dtype: object

In [172]:
wave = data_table.SwH * data_table.SwP
data_table['Wave'] = wave

In [173]:
article_read.WWD.unique()

array(['degT', 'SSW', 'S', 'SW', 'ESE', 'SSE', 'SE', 'E', 'WSW', 'ENE',
       'N', 'W', 'NNE', 'WNW', 'NE', 'NW', 'NNW'], dtype=object)

In [174]:
wwd = pd.DataFrame({"WWD": [ 'SW', 'SSW', 'S', 'ESE', 'SSE', 'SE', 'E', 'WSW', 'ENE',
       'N', 'W', 'NNE', 'WNW', 'NE', 'NW', 'NNW' ], "scoreWWD": [1, 1, 3 ,1 ,1 ,4 ,5 ,1 ,1 ,1 ,1 ,1 , 1, 4,1,1]})
wwd.head()

,WWD,scoreWWD
0,SW,1
1,SSW,1
2,S,3
3,ESE,1
4,SSE,1


In [175]:
data_table =pd.merge(data_table, wwd)
data_table.head()

,SwH,SwP,WWH,WWP,SwD,WWD,Date,Wave,scoreWWD
0,1.6,12.1,0.6,4.8,W,SSW,2019-03-21,19.36,1
1,1.8,10.8,0.6,3.7,W,SSW,2019-03-21,19.44,1
2,2.1,12.1,0.5,3.6,W,SSW,2019-03-21,25.41,1
3,2.1,12.1,0.5,5.0,W,SSW,2019-03-21,25.41,1
4,2.0,13.8,0.5,4.5,W,SSW,2019-03-21,27.60,1


In [176]:
def scoreWave(column):
    if column > 30 :
        score = 5
    elif column >= 25 :
        score = 4
    elif column >= 20 :
        score = 3
    elif column >= 11 :
        score = 2
    else:
        score =1
    return score 

In [177]:
def scoreSwp(column):
    if column >= 16 :
        score = 5
    elif column >= 12 :
        score = 4
    elif column >= 10 :
        score = 4
    else:
        score =1
    return score 

In [178]:
data_table['scoreWave'] = data_table['Wave'].map(scoreWave)

In [179]:
data_table['scoreSwP'] = data_table['SwP'].map(scoreSwp)

In [186]:
data_table['Score'] = data_table.iloc[:, 8:11].sum(axis=1)

In [187]:
data_length = len(data_table)-1
data_length

21

In [188]:
data_table

,SwH,SwP,WWH,WWP,SwD,WWD,Date,Wave,scoreWWD,scoreWave,scoreSwP,Score
0,1.6,12.1,0.6,4.8,W,SSW,2019-03-21,19.36,1,2,4,7
1,1.8,10.8,0.6,3.7,W,SSW,2019-03-21,19.44,1,2,4,7
2,2.1,12.1,0.5,3.6,W,SSW,2019-03-21,25.41,1,4,4,9
3,2.1,12.1,0.5,5.0,W,SSW,2019-03-21,25.41,1,4,4,9
4,2.0,13.8,0.5,4.5,W,SSW,2019-03-21,27.60,1,4,4,9
5,1.9,12.9,0.5,3.3,W,SSW,2019-03-21,24.51,1,3,4,8
6,1.9,12.9,0.8,3.3,W,SSW,2019-03-21,24.51,1,3,4,8
7,1.9,13.8,0.7,3.3,WNW,SSW,2019-03-21,26.22,1,4,4,9
8,2.1,13.8,0.3,4.5,W,SSW,2019-03-21,28.98,1,4,4,9
9,1.8,12.1,0.6,3.8,WSW,S,2019-03-21,21.78,3,3,4,10


In [189]:
data_table.apply(np.sum, axis=0)

SwH                                                       42.4
SwP                                                      278.5
WWH                                                       12.3
WWP                                                       94.3
SwD                       WWWWWWWWNWWWSWWWSWWWSWWWWSWWNWWWWNWW
WWD            SSWSSWSSWSSWSSWSSWSSWSSWSSWSSSSSSSWSWSWSWSWSWSW
Date         2019-03-212019-03-212019-03-212019-03-212019-0...
Wave                                                    537.46
scoreWWD                                                    34
scoreWave                                                   75
scoreSwP                                                    88
Score                                                      197
dtype: object

In [190]:
totalscore = data_table['Score'].sum()/(len(data_table)-1)
totalscore

9.380952380952381

In [191]:
day_score = round(totalscore)
day_score

9.0

# Test

# UI